In [ ]:
import os
import json
import requests
import time 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime

from functions import load_api_key, plotting

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

# Foreign Workers (FW) or Foreign Talents (FT)

In [ ]:
FT_df = pd.read_excel('inputs/foreign-workforce-numbers.xlsx')
FT_df.columns = FT_df.iloc[0]  
FT_df = FT_df.T
FT_df.columns = FT_df.iloc[0]  
FT_df = FT_df.iloc[1:]
FT_df = FT_df.reset_index(drop=True)  

In [ ]:
FT_df.columns

In [ ]:
FT_df

In [ ]:
FT_df = FT_df[['Pass Type', 'Employment Pass (EP)', 'S Pass', 'Work Permit (Total)', '- Work Permit (Migrant Domestic Workers)',
       '- Work Permit (Construction, Marine Shipyard and Process)',
       'Other Work Passes2', 'Total Foreign Workforce']]
FT_df = FT_df.rename(columns= {'- Work Permit (Construction, Marine Shipyard and Process)' : 'Work Permit (Construction, Marine Shipyard and Process)', 
                               'Other Work Passes2':'Other Work Passes', 
                              '- Work Permit (Migrant Domestic Workers)' : 'Work Permit (Migrant Domestic Workers)'})

In [ ]:
for col in FT_df.columns:
    if col == 'Pass Type':
        continue
    else:
        FT_df[col] = FT_df[col].astype(int)

In [ ]:
FT_df['datetime'] = pd.to_datetime(FT_df['Pass Type'], errors = 'coerce')
FT_df['year'] = FT_df['datetime'].dt.year

In [ ]:
FT_df.dtypes

In [ ]:
FT_df

In [ ]:
FT_df['Total Work Permit (excluding Migrant Domestic Workers)'] = FT_df['Work Permit (Total)'] - FT_df['Work Permit (Migrant Domestic Workers)']

In [ ]:
FT_df

In [ ]:
collapsed_FT_df = pd.melt(FT_df, id_vars=["year"], var_name="category", value_name="number_of_people")

In [ ]:
order = ['Employment Pass (EP)', 'S Pass', 'Work Permit (Construction, Marine Shipyard and Process)', 'Other Work Passes', 'Total Work Permit (excluding Migrant Domestic Workers)', 'Total Foreign Workforce']
df = collapsed_FT_df.loc[(collapsed_FT_df['category'].isin(order)) & (collapsed_FT_df['year'] >= 2021)].reset_index(drop = True)
df['number_of_people'] = df['number_of_people'].astype(int)
plotting('FT vs time', 'barplot', df, 'category', 'number_of_people', 'year', desired_order = order)

# Latest population figures

In [ ]:
population_df = pd.read_csv('inputs/Indicators On Population.csv')
population_df.columns = population_df.iloc[7]
population_df = population_df.iloc[8:15]
population_df =population_df.T
population_df.columns = population_df.iloc[0]
population_df = population_df.iloc[1:6]
population_df = population_df .reset_index()
population_df = population_df.rename(columns = {7: 'year'})

In [ ]:
population_df

In [ ]:
collapsed_population_df = pd.melt(population_df, id_vars=["year"], var_name="category", value_name="number_of_people")
collapsed_population_df['year'] = collapsed_population_df['year'].astype(int)
collapsed_population_df['number_of_people'] = collapsed_population_df['number_of_people'].astype(float)

In [ ]:
non_resident_population_df = population_df[['year', 'Non-Resident Population (Number)']].copy()
non_resident_population_df['year'] = non_resident_population_df['year'].astype(int)
non_resident_population_df['Non-Resident Population (Number)'] = non_resident_population_df['Non-Resident Population (Number)'].astype(int)
non_resident_population_df = non_resident_population_df.sort_values(by = 'year')
non_resident_population_df['number_of_people'] = non_resident_population_df['Non-Resident Population (Number)'].pct_change() * 100
non_resident_population_df['category'] = 'Non-Resident Growth (Per Cent)'
collapsed_population_df = pd.concat([collapsed_population_df, non_resident_population_df[['year', 'number_of_people', 'category']]])

In [ ]:
order = ['Total Population (Number)', 'Resident Population (Number)',
       'Singapore Citizen Population (Number)',
       'Permanent Resident Population (Number)',
       'Non-Resident Population (Number)']
df = collapsed_population_df.loc[(collapsed_population_df['category'].isin(order)) & (collapsed_population_df['year'] >= 2022)].reset_index(drop = True)
plotting('Population vs time', 'barplot', df, 'category', 'number_of_people', 'year', desired_order = order)

In [ ]:
order = ['Non-Resident Growth (Per Cent)', 'Resident Population Growth (Per Cent)', 'Total Population Growth (Per Cent)']
df = collapsed_population_df.loc[(collapsed_population_df['category'].isin(order)) & (collapsed_population_df['year'] >= 2022)].reset_index(drop = True)
plotting('Population growth vs time', 'barplot', df, 'category', 'number_of_people', 'year', desired_order = order)